In [ ]:
import torch.optim as optim
from datetime import datetime

from transformer import Transformer
from preprocessing import *
from utils import *
from features import *

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.set_per_process_memory_fraction(0.)
else:
    device = torch.device("cpu")

In [ ]:
raw_data = pd.read_csv('consumption_and_temperatures.csv')
raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])

seq_len = 48
scale_output = True
target_column = 'NO1_consumption'

features_to_add = [
    (   
        pick_location_data,
        { 'loc': [1] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {   
            "shift_min": 24,
            "shift_max": 24,
            "column_to_shift": "NO1_consumption",
            "new_column_name": "consum"
        },
    ),
    (
        add_hour_columns,
        {}
    )
]

forecast_len=24


In [ ]:
(X_train, y_train), (X_val, y_val), (X_test, y_test), (scalerInputMethod, scalerOutputMethod), df_target = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add,
        seq_len=seq_len,
        forecast_len=24,
        scale_output=scale_output
    )

model = Transformer(input_size=X_train.shape[2])
model.to(device)

# path = "Transformer-2024-03-20-loss-0.035382744"
# path = "models/" + path
# model.load_state_dict(torch.load(path))
# model.to(device)

lr = 1e-3
num_epochs=100

loss_func = nn.MSELoss()# .to(device)
opt = optim.Adam(model.parameters(), lr=lr)

model, losses, val_loss, opt = fit(
    model, 
    train_set=(X_train, y_train),
    val_set=(X_val, y_val),
    opt=opt,
    loss_func=loss_func,
    num_epochs=num_epochs,
    device=device,
    batch_size=30
)

path = f'models/Transformer-{str(datetime.now().date())}-loss-{str(losses[len(losses)-1])}'
torch.save(model.state_dict(), path)

plt.plot(losses, label="train loss")
plt.plot(val_loss, label="val loss")
plt.legend()

y_test, y_pred = predict(model, scalerOutputMethod, (X_test, y_test))
plot_error_by_hour_for_test_set(y_test, y_pred, start_hour=df_target['timestamp'].dt.hour.iloc[seq_len-1])

for k in range(5):
    make_forecast(y_pred=y_pred, df_target=df_target, seq_len=seq_len)